In [122]:
import requests
from collections import Counter



In [123]:
year = 2023

In [124]:
def findTeamKeys():
    mlbTeamKeys = {}
    url = "https://statsapi.mlb.com/api/v1/teams/"
    response = requests.get(url)
    data = response.json()
    for team in data['teams']:

        if team.get('sport', {}).get('name', '') == "Major League Baseball":
            print()
            mlbTeamKeys[team.get('name', '')] = team.get('id', -1)
    return mlbTeamKeys

In [125]:
mlbTeamKeys = findTeamKeys()

Step 1-2:

In [126]:
for index, team in enumerate(sorted(mlbTeamKeys)):
    print(index+1, team)
teamSelection = sorted(mlbTeamKeys)[int(input("Select a number to select what team you would like: "))-1]
print(teamSelection)

1 Arizona Diamondbacks
2 Atlanta Braves
3 Baltimore Orioles
4 Boston Red Sox
5 Chicago Cubs
6 Chicago White Sox
7 Cincinnati Reds
8 Cleveland Guardians
9 Colorado Rockies
10 Detroit Tigers
11 Houston Astros
12 Kansas City Royals
13 Los Angeles Angels
14 Los Angeles Dodgers
15 Miami Marlins
16 Milwaukee Brewers
17 Minnesota Twins
18 New York Mets
19 New York Yankees
20 Oakland Athletics
21 Philadelphia Phillies
22 Pittsburgh Pirates
23 San Diego Padres
24 San Francisco Giants
25 Seattle Mariners
26 St. Louis Cardinals
27 Tampa Bay Rays
28 Texas Rangers
29 Toronto Blue Jays
30 Washington Nationals
Arizona Diamondbacks


Step 3

In [127]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Get the current date
current_date = datetime.now().date()

# Calculate the end date as the previous day
end_date = current_date - timedelta(days=1)

# Calculate the beginning date as exactly one month before the end date
beginning_date = end_date - relativedelta(months=1)

# Print the results
print("End Date:", end_date)
print("Beginning Date:", beginning_date)


#Pull json to get the game id's from startdate-enddate
url = f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&teamId={mlbTeamKeys[teamSelection]}&startDate={beginning_date}&endDate={end_date}"
response = requests.get(url)
data = response.json()

#Assign each date to their respective game key, if there was a double-header played on that date then both id's will be added
gameKeyDict = {}
for dateDict in data['dates']:

    if dateDict['totalGames'] == 1:
        gameKeyDict[dateDict['date']] = [dateDict['games'][0]['gamePk']]
    else:
        gameKeyDict[dateDict['date']] = []
        for game in range(dateDict['totalGames']):
            gameKeyDict[dateDict['date']].append(dateDict['games'][game]['gamePk'])


gameKeyDict



End Date: 2023-05-24
Beginning Date: 2023-04-24


{'2023-04-24': [718438],
 '2023-04-25': [718422],
 '2023-04-26': [718418],
 '2023-04-28': [718390],
 '2023-04-29': [718366],
 '2023-04-30': [718358],
 '2023-05-02': [718342],
 '2023-05-03': [718325],
 '2023-05-05': [718296],
 '2023-05-06': [718282],
 '2023-05-07': [718263],
 '2023-05-08': [718249],
 '2023-05-09': [718236],
 '2023-05-10': [718226],
 '2023-05-11': [718215],
 '2023-05-12': [718199],
 '2023-05-13': [718184],
 '2023-05-14': [718169],
 '2023-05-15': [718160],
 '2023-05-16': [718146],
 '2023-05-17': [718139],
 '2023-05-19': [718118],
 '2023-05-20': [718101],
 '2023-05-21': [718091],
 '2023-05-22': [718073],
 '2023-05-23': [718060],
 '2023-05-24': [718043]}

In [128]:
playerIDDict = {}
for team in mlbTeamKeys.keys():
    url = f"https://statsapi.mlb.com/api/v1/teams/{mlbTeamKeys[team]}/roster/fullRoster?season=2023"
    response = requests.get(url)
    data = response.json()
    playerIDDict[team] = {'hitters' : {}, 'pitchers' : {}}
    for player in data["roster"]:
        playerName = player['person']['fullName']
        playerID = player['person']['id']
        playerStatus = player['status']['code']
        playerPosition  = player['position']['abbreviation']


        if playerStatus == 'A':
            if playerPosition == 'P':
                playerIDDict[team]['pitchers'][playerName] = playerID
            else:
                playerIDDict[team]['hitters'][playerName] = playerID
            
            # playerURL = f"https://statsapi.mlb.com/api/v1/people?personIds={playerID}&season=2023&hydrate=stats(type=gameLog,season=2023,gameType=R)"
            # response = requests.get(playerURL)
            # data = response.json()

            # if "stats" in data['people'][0].keys():
            #     if player['position']['abbreviation'] == 'P':
            #         playerIDDict[team]['pitchers'][player['person']["fullName"]]= playerID
            #     else:
            #         playerIDDict[team]['hitters'][player['person']["fullName"]]= playerID




In [129]:
print(mlbTeamKeys)

{'Los Angeles Angels': 108, 'Arizona Diamondbacks': 109, 'Baltimore Orioles': 110, 'Boston Red Sox': 111, 'Chicago Cubs': 112, 'Cincinnati Reds': 113, 'Cleveland Guardians': 114, 'Colorado Rockies': 115, 'Detroit Tigers': 116, 'Houston Astros': 117, 'Kansas City Royals': 118, 'Los Angeles Dodgers': 119, 'Washington Nationals': 120, 'New York Mets': 121, 'Oakland Athletics': 133, 'Pittsburgh Pirates': 134, 'San Diego Padres': 135, 'Seattle Mariners': 136, 'San Francisco Giants': 137, 'St. Louis Cardinals': 138, 'Tampa Bay Rays': 139, 'Texas Rangers': 140, 'Toronto Blue Jays': 141, 'Minnesota Twins': 142, 'Philadelphia Phillies': 143, 'Atlanta Braves': 144, 'Chicago White Sox': 145, 'Miami Marlins': 146, 'New York Yankees': 147, 'Milwaukee Brewers': 158}


In [130]:
import requests
import pandas as pd
import numpy as np


def pullRoster(gameID):
    # Fetch the data
    url = f'https://statsapi.mlb.com/api/v1/game/{gameID}/boxscore'
    response = requests.get(url)
    data = response.json()

    # Prepare the teams
    teams = ['home', 'away']

    # Prepare an empty dataframe for the box score
    box_score = pd.DataFrame()

    # Loop over the teams
    for team in teams:
        # Get the team data
        team_data = data['teams'][team]

        # Get the team name
        team_name = team_data['team']['name']

        # Get the players
        players = team_data['players']

        # Loop over the players
        for player_id, player_data in players.items():
            # Skip if the player is a pitcher
            if player_data['position']['abbreviation'] == 'P':
                continue

            # Prepare a row for the player
            player_row = pd.DataFrame({
                'Team': [team_name],
                'Player': [player_data['person']['fullName']],
                'Batting Order' : [player_data.get('battingOrder', 'N/A')],
                'Position': [player_data['position']['abbreviation']],
                'AB': [player_data['stats']['batting'].get('atBats', 'N/A')],
                'R': [player_data['stats']['batting'].get('runs', 'N/A')],
                'H': [player_data['stats']['batting'].get('hits', 'N/A')],
                'RBI': [player_data['stats']['batting'].get('rbi', 'N/A')],
                'BB': [player_data['stats']['batting'].get('baseOnBalls', 'N/A')],
                'SO': [player_data['stats']['batting'].get('strikeOuts', 'N/A')],
                'AVG': [player_data['seasonStats']['batting'].get('avg', 'N/A')],
            })
            player_row['Batting Order'] = pd.to_numeric(player_row['Batting Order'], errors='coerce').astype('Int64')

            # Add the row to the box score
            box_score = pd.concat([box_score, player_row], ignore_index=True)

    # Set the column order
    columns = ['Team', 'Player', 'Batting Order', 'Position', 'AB', 'R', 'H', 'RBI', 'BB', 'SO', 'AVG']
    box_score = box_score[columns]




    # Display the box score
    box_score.replace('N/A', np.nan, inplace=True)
    box_score.dropna(inplace=True)

    df = pd.DataFrame(box_score)
    rosterdf = box_score[box_score['Team'] == teamSelection]
    rosterdf = rosterdf.sort_values('Batting Order')

    for index, value in rosterdf['Batting Order'].items():
        if value %100:
            rosterdf.loc[index, 'Position'] = 'PH'
    rosterdf = rosterdf.loc[rosterdf['Position'] != 'PH']

    return rosterdf

In [133]:
rosterSample = []

for date in list(gameKeyDict.keys())[-5:]:
    rosterdf = pullRoster(gameKeyDict[date][0])
    rosterSample.append([player for player in rosterdf['Player'].to_list()])
rosterSample
flattened_list = [item for sublist in rosterSample for item in sublist]
flattened_list
counts = Counter(flattened_list)
result = [element for element, count in counts.items() if count >= 3]


roster = {teamSelection : result}


roster




{'Arizona Diamondbacks': ['Josh Rojas',
  'Ketel Marte',
  'Corbin Carroll',
  'Christian Walker',
  'Lourdes Gurriel Jr.',
  'Pavin Smith',
  'Dominic Fletcher',
  'Gabriel Moreno',
  'Geraldo Perdomo']}

In [132]:
rosterdf

,Team,Player,Batting Order,Position,AB,R,H,RBI,BB,SO,AVG
15,Arizona Diamondbacks,Ketel Marte,100,2B,5.0,1.0,2.0,0.0,0.0,0.0,.279
20,Arizona Diamondbacks,Corbin Carroll,200,LF,4.0,1.0,1.0,0.0,1.0,1.0,.286
25,Arizona Diamondbacks,Emmanuel Rivera,300,3B,4.0,1.0,1.0,3.0,1.0,0.0,.323
22,Arizona Diamondbacks,Christian Walker,400,1B,4.0,0.0,0.0,0.0,1.0,1.0,.249
23,Arizona Diamondbacks,Evan Longoria,500,DH,2.0,1.0,1.0,2.0,1.0,1.0,.221
24,Arizona Diamondbacks,Pavin Smith,600,RF,4.0,0.0,0.0,0.0,1.0,1.0,.230
16,Arizona Diamondbacks,Nick Ahmed,700,SS,4.0,0.0,0.0,0.0,0.0,3.0,.238
19,Arizona Diamondbacks,Dominic Fletcher,800,CF,3.0,0.0,0.0,0.0,1.0,1.0,.308
13,Arizona Diamondbacks,Jose Herrera,900,C,4.0,1.0,1.0,0.0,0.0,2.0,.227
